In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Word Count") \
    .getOrCreate()

22/10/24 11:04:34 WARN Utils: Your hostname, Rajivs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.69 instead (on interface en0)
22/10/24 11:04:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/24 11:04:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [23]:
destination_dir = "/Users/rajivsunar/Documents/projects/premier_league_data/data"
fixtures_df = spark.read.json(destination_dir + "/fixtures.json")
teams_df = spark.read.json(destination_dir + "/teams.json")
events_df = spark.read.json(destination_dir + "/events.json")

In [14]:
list(fixtures_df.drop("stats").printSchema())

root
 |-- code: long (nullable = true)
 |-- event: long (nullable = true)
 |-- finished: boolean (nullable = true)
 |-- finished_provisional: boolean (nullable = true)
 |-- id: long (nullable = true)
 |-- kickoff_time: string (nullable = true)
 |-- minutes: long (nullable = true)
 |-- provisional_start_time: boolean (nullable = true)
 |-- pulse_id: long (nullable = true)
 |-- started: boolean (nullable = true)
 |-- team_a: long (nullable = true)
 |-- team_a_difficulty: long (nullable = true)
 |-- team_a_score: long (nullable = true)
 |-- team_h: long (nullable = true)
 |-- team_h_difficulty: long (nullable = true)
 |-- team_h_score: long (nullable = true)



TypeError: 'NoneType' object is not iterable

In [8]:
fixtures_df.filter(fixtures_df.event == 11).select("code", "event", "team_a", "team_h").show()

+-------+-----+------+------+
|   code|event|team_a|team_h|
+-------+-----+------+------+
|2292911|   11|     5|     4|
|2292914|   11|     7|    10|
|2292912|   11|     3|     9|
|2292919|   11|    16|    20|
|2292918|   11|     8|    18|
|2292910|   11|     6|     2|
|2292913|   11|     1|    11|
|2292916|   11|    15|    14|
|2292917|   11|    19|    17|
|2292915|   11|    13|    12|
+-------+-----+------+------+



In [27]:
f_a = fixtures_df.join(teams_df, fixtures_df["team_a"] == teams_df["id"]).withColumnRenamed("name", "team_a_name")\
    .select("team_a_name", "event", "team_h", "team_h_score", "team_a_score", "finished", "minutes", "kickoff_time", "started")\
    .join(teams_df, fixtures_df["team_h"] == teams_df["id"]).withColumnRenamed("name", "team_h_name")\
    .select("team_a_name", "event", "team_h_name", "team_a_score", "team_h_score",  "finished", "minutes", "kickoff_time", "started")
# f_h = f_a.join(teams_df, fixtures_df["team_h"] == teams_df["code"]).withColumnRenamed("name", "team_h_name")
f_a.filter(f_a.event == 11).show()

# f_h = f_a.join(teams_df, f_a["team_h"] == teams_df["id"]).withColumnRenamed("name", "team_h_name")\
#     .select("team_a_name", "event", "team_h_name", "team_a_score", "team_h_score",  "finished", "minutes", "kickoff_time", "started")
# f_h.filter(f_h.event == 13).show()
# f_a.select("event", "team_a", "team_h", "team_a_name").filter(f_a["event" == 11]).show()
# f_h = f_a.join(teams_df, f_a["team_h"] == teams_df["code"])

+--------------+-----+-----------+------------+------------+--------+-------+--------------------+-------+
|   team_a_name|event|team_h_name|team_a_score|team_h_score|finished|minutes|        kickoff_time|started|
+--------------+-----+-----------+------------+------------+--------+-------+--------------------+-------+
|      Brighton|   11|  Brentford|           0|           2|    true|     90|2022-10-14T19:00:00Z|   true|
|Crystal Palace|   11|  Leicester|           0|           0|    true|     90|2022-10-15T11:30:00Z|   true|
|   Bournemouth|   11|     Fulham|           2|           2|    true|     90|2022-10-15T14:00:00Z|   true|
| Nott'm Forest|   11|     Wolves|           0|           1|    true|     90|2022-10-15T14:00:00Z|   true|
|       Everton|   11|      Spurs|           0|           2|    true|     90|2022-10-15T16:30:00Z|   true|
|       Chelsea|   11|Aston Villa|           2|           0|    true|     90|2022-10-16T13:00:00Z|   true|
|       Arsenal|   11|      Leeds|   

In [26]:
# joining gameweek
events = events_df.select("id", "name")
f_e = f_h.join(events, f_h["event"] == events["id"]).drop("id", "event")
f_e.show()

+--------------+--------------+------------+------------+--------+-------+--------------------+-------+----------+
|   team_a_name|   team_h_name|team_a_score|team_h_score|finished|minutes|        kickoff_time|started|      name|
+--------------+--------------+------------+------------+--------+-------+--------------------+-------+----------+
|       Arsenal|Crystal Palace|           2|           0|    true|     90|2022-08-05T19:00:00Z|   true|Gameweek 1|
|     Liverpool|        Fulham|           2|           2|    true|     90|2022-08-06T11:30:00Z|   true|Gameweek 1|
|   Aston Villa|   Bournemouth|           0|           2|    true|     90|2022-08-06T14:00:00Z|   true|Gameweek 1|
|        Wolves|         Leeds|           1|           2|    true|     90|2022-08-06T14:00:00Z|   true|Gameweek 1|
| Nott'm Forest|     Newcastle|           0|           2|    true|     90|2022-08-06T14:00:00Z|   true|Gameweek 1|
|   Southampton|         Spurs|           1|           4|    true|     90|2022-0

In [44]:
eleven = fixtures_df.filter(fixtures_df.event == 11).select("event", "team_a", "team_h")
eleven.show()

+-----+------+------+
|event|team_a|team_h|
+-----+------+------+
|   11|     5|     4|
|   11|     7|    10|
|   11|     3|     9|
|   11|    16|    20|
|   11|     8|    18|
|   11|     6|     2|
|   11|     1|    11|
|   11|    15|    14|
|   11|    19|    17|
|   11|    13|    12|
+-----+------+------+



In [45]:
eleven.join(teams_df, eleven["team_a"] == teams_df["id"]).select("event", "team_a", "team_h").show()

+-----+------+------+
|event|team_a|team_h|
+-----+------+------+
|   11|     5|     4|
|   11|     7|    10|
|   11|     3|     9|
|   11|    16|    20|
|   11|     8|    18|
|   11|     6|     2|
|   11|     1|    11|
|   11|    15|    14|
|   11|    19|    17|
|   11|    13|    12|
+-----+------+------+

